In [2]:
# --- Path bootstrap so "from utils.config import ..." works no matter where Jupyter's CWD is ---
import sys
from pathlib import Path

def add_codebase_to_path(marker_dir_name="code_base"):
    cwd = Path.cwd().resolve()
    # walk up until we find "code_base" that contains utils/config.py
    for p in [cwd, *cwd.parents]:
        if p.name == marker_dir_name and (p / "utils" / "config.py").exists():
            sys.path.insert(0, str(p))  # add code_base itself
            return p
        # if we're at repo root and "code_base" is a child, handle that too
        if (p / marker_dir_name / "utils" / "config.py").exists():
            sys.path.insert(0, str(p / marker_dir_name))
            return p / marker_dir_name
    raise RuntimeError("Could not locate code_base/ with utils/config.py")

CODEBASE_DIR = add_codebase_to_path()
print("Using code base at:", CODEBASE_DIR)


Using code base at: /storage/ice1/1/0/vchopra37/projects/edge_glass/code_base


In [3]:
import os, sys, json, platform, torch
import transformers
from utils.config import load_config, select_device, select_dtype

### Cell 1 — Imports & env echo

In [5]:
print("Python:", platform.python_version())
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
print("HF_HOME:", os.environ.get("HF_HOME"))
print("TORCH_HOME:", os.environ.get("TORCH_HOME"))


Python: 3.12.5
Torch: 2.9.0+cu128
Transformers: 4.57.1
CUDA available: True
GPU: NVIDIA H200
HF_HOME: /home/hice1/vchopra37/scratch/hf_home
TORCH_HOME: /home/hice1/vchopra37/scratch/torch


### Cell 2 — Load config & inspect resolved paths

In [7]:
cfg = load_config()  # picks code_base/configs/base.yaml by default
device = select_device(cfg)
dtype = select_dtype(cfg)

resolved = cfg.paths.resolve()
print("Device:", device)
print("DType:", dtype)
print("Resolved paths:", {k:str(v) for k,v in resolved.items()})
cfg


Device: cuda
DType: torch.float16
Resolved paths: {'data': '/storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/data', 'out': '/storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/experiments', 'embeds': '/storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/experiments/embeddings'}


Config(device='cuda', dtype='fp16', seeds=Seeds(python=1337, torch=1337), paths=Paths(data='./data', out='./experiments', embeds='./experiments/embeddings'), encoders=Encoders(vision='openai/clip-vit-base-patch32', text='sentence-transformers/all-MiniLM-L6-v2', audio='openai/whisper-small'), cfg_path=PosixPath('/storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/configs/base.yaml'))

### Cell 3 — Create a run artifact under code_base/experiments/

In [8]:
from pathlib import Path
run_info = {
    "phase": "0_bootstrap",
    "device": str(device),
    "dtype": cfg.dtype,
    "encoders": {
        "vision": cfg.encoders.vision,
        "text": cfg.encoders.text,
        "audio": cfg.encoders.audio,
    },
    "cfg_path": str(cfg.cfg_path),
}
runs_dir = cfg.paths.resolve()["out"] / "runs"
runs_dir.mkdir(parents=True, exist_ok=True)
with open(runs_dir / "phase0_bootstrap.json", "w") as f:
    json.dump(run_info, f, indent=2)
run_info


{'phase': '0_bootstrap',
 'device': 'cuda',
 'dtype': 'fp16',
 'encoders': {'vision': 'openai/clip-vit-base-patch32',
  'text': 'sentence-transformers/all-MiniLM-L6-v2',
  'audio': 'openai/whisper-small'},
 'cfg_path': '/storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/configs/base.yaml'}

### Cell 4 — Micro GPU test (optional)

In [9]:
if torch.cuda.is_available():
    x = torch.randn(1024, 1024, device=device, dtype=dtype if dtype!=torch.float16 or torch.cuda.is_available() else torch.float32)
    y = x @ x.T
    print("Matmul OK:", y.shape)
else:
    print("CPU mode. Matmul OK (small):", (torch.randn(64,64) @ torch.randn(64,64)).shape)


Matmul OK: torch.Size([1024, 1024])
